In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import json
from fetch_statuses import get_lot_names

In [ ]:
with open('./lotRecords.json') as f:
    records = json.load(f)
flat_records = [dict(time=time, id=lot_id, status=status) for (time, lots) in records.items() for (lot_id, status) in lots.items()]
df = pd.DataFrame(flat_records)
df['time'] = pd.to_datetime(df['time'], unit='s').dt.round('30min')
df

In [ ]:
names = pd.Series(get_lot_names())
names

In [ ]:
df2 = pd.pivot(df, index='id', columns='time').status
no_status = ((df2 == 'na' )).all(axis='columns')
just_active = ((df2 == 'active' )).all(axis='columns')
lot_ids_without_status = set(df2[no_status | just_active].index)

In [ ]:


# df2 = df2[~no_status & ~just_active]

time_columns = df2.columns

df2['grade'] = df2.apply(lambda row: row.map(dict(available=0, few=0.8, full=1, na=0, active=0)).sum(), axis='columns')
df2 = df2.sort_values(by=['grade'], ascending=False)
df2['name'] = names
df2 = df2.reindex(['name', 'grade'] + list(time_columns), axis='columns')

In [ ]:

numeric_data = df2.drop(['grade', 'name'], axis='columns').applymap(lambda x: {'available':0, 'few': 0.8, 'full': 2, 'na': -1, 'active': -1, 'closed': -1}[x]).to_numpy()
plt.imshow(numeric_data)
plt.show()

In [ ]:
nice_json_dict = dict()
for column in df2:
    if column in ['grade', 'name']:
        continue
    key = column.isoformat()
    values = df2[column].to_dict()
    nice_json_dict[key] = values

with open('parsed_lot_records.json', 'w')  as f:
    json.dump(nice_json_dict, f)

